In [1]:
import pandas as ps
import numpy as np 
import os
import re

In [2]:
os.listdir("data")

['Flore du Cameroun tome 10 1970 Ombellales',
 'Flore du Cameroun tome 10 1970 Ombellales.pdf',
 'Flore du Cameroun tome 12 1972 Loganiaceae',
 'Flore du Cameroun tome 12 1972 Loganiaceae.pdf',
 'Flore du Cameroun tome 16 1973 Sapindaceae',
 'Flore du Cameroun tome 16 1973 Sapindaceae.pdf',
 'Flore du Cameroun tome 17 1974 Amaranthaceae',
 'Flore du Cameroun tome 17 1974 Amaranthaceae.pdf',
 'Flore du Cameroun tome 2 1964 Sapotacees',
 'Flore du Cameroun tome 2 1964 Sapotacees.pdf',
 'Flore du Cameroun tome 6 1967 Cucurbitaceae',
 'Flore du Cameroun tome 6 1967 Cucurbitaceae.pdf',
 'Flore du Cameroun tome 9 1970 Caesalpinioidae',
 'Flore du Cameroun tome 9 1970 Caesalpinioidae.pdf']

In [3]:
folder = []
for item in os.listdir('data'):
    if(len(item.split('.')) == 1):
        folder.append(item)

In [4]:
folder

['Flore du Cameroun tome 10 1970 Ombellales',
 'Flore du Cameroun tome 12 1972 Loganiaceae',
 'Flore du Cameroun tome 16 1973 Sapindaceae',
 'Flore du Cameroun tome 17 1974 Amaranthaceae',
 'Flore du Cameroun tome 2 1964 Sapotacees',
 'Flore du Cameroun tome 6 1967 Cucurbitaceae',
 'Flore du Cameroun tome 9 1970 Caesalpinioidae']

In [ ]:
famille =[]
famille_ = []
tailles = []

#for f in folder:
path = 'data/' + 'Flore du Cameroun tome 12 1972 Loganiaceae'
print("=============================================================")
print(path)
for i in range(len(os.listdir(path))):
    with open(path+'/'+ str(i+1) +'.txt', 'r') as fp:
        lines = fp.readlines()
        #print("=================")
        for j in range(len(lines)):
            l = lines[j].strip()
            l = l.replace('  ',' ')
            #print(l)
            l = f_remove_accents(l)
            x = re.search("^cle.*espece", l)
            if(x):
                print(l)

In [ ]:
path = 'data/' + 'Flore du Cameroun tome 9 1970 Caesalpinioidae'
print("=============================================================")
print(path)
for i in range(len(os.listdir(path))):
    with open(path+'/'+ str(i+1) +'.txt', 'r') as fp:
        lines = fp.readlines()
        #print("=================")
        for j in range(len(lines)):
            l = lines[j].strip()
            l = l.replace('  ',' ')
            #print(l)
            l = f_remove_accents(l)
            x = re.search("^cle.*espece", l)
            if(x):
                print(lines[j])

In [ ]:
path = 'data/' + 'Flore du Cameroun tome 2 1964 Sapotacees'
print("=============================================================")
print(path)
for i in range(len(os.listdir(path))):
    with open(path+'/'+ str(i+1) +'.txt', 'r') as fp:
        lines = fp.readlines()
        #print("=================")
        for j in range(len(lines)):
            l = lines[j].strip()
            l = l.replace('  ',' ')
            #print(l)
            l = f_remove_accents(l)
            x = re.search("^cle.*espece", l)
            if(x):
                print(lines[j])

In [6]:
def f_remove_accents(old):
    """
    Removes common accent characters, lower form.
    Uses: regex.
    """
    new = old.lower()
    new = re.sub(r'[àáâãäå]', 'a', new)
    new = re.sub(r'[èéêë]', 'e', new)
    new = re.sub(r'[ìíîï]', 'i', new)
    new = re.sub(r'[òóôõö]', 'o', new)
    new = re.sub(r'[ùúûü]', 'u', new)
    return new
f_remove_accents("CLÉ DES ESPÈCES CAMEROUNAISES ET GABONAISES")


'cle des especes camerounaises et gabonaises'

In [ ]:
"ESPÈCES".lower()

In [37]:
famille =[]
famille_ = []
tailles = []

#for f in folder:
path = 'data/' + 'Flore du Cameroun tome 12 1972 Loganiaceae'
print("=============================================================")
print(path)
for i in range(len(os.listdir(path))):
    with open(path+'/'+ str(i+1) +'.txt', 'r') as fp:
        lines = fp.readlines()
        #print("=================")
        for j in range(len(lines)):
            l = lines[j].strip()
            l = l.replace('  ',' ')
            #print(l)
            l = f_remove_accents(l)
            x = re.search("^cle.*espece", l)
            if(x):
                print(i+1)
                print(l)

data/Flore du Cameroun tome 12 1972 Loganiaceae
9
cle des especes camerounaises et gabonaises
30
cle des especes camerounaises et gabonaises
49
cle des especes camerounaises et gabonaises
54
cle des especes camerounaises et gabonaises


In [78]:
with open(path+'/'+ str(9) +'.txt', 'r') as fp:
    lines = fp.readlines()
    #print("=================")
    for j in range(len(lines)):
        #print(lines[j])
        l = lines[j].strip()
        l = l.replace('  ',' ')
        #print(l)
        l = f_remove_accents(l)
        x = re.findall("\d\.\s\w\.\s\w+\.$", l)
        if(x):
            print(x)
        #   print(l)

['2. a. laxiflora.']
['8. a. vogelii.']
['1. a. djalonensis.']
['1. a. djalonensis.']
['3. a. liebrechtsiana.']
['7. a. schweinfurthii.']
['6. a. scandens.']
['4. a. microphylla.']
['5. a. obanensis.']


In [123]:
with open(path+'/'+ str(61) +'.txt', 'r') as fp:
    lines = fp.readlines()
    #print("=================")
    for j in range(len(lines)):
        #print(lines[j])
        l = lines[j].strip()
        l = l.replace('  ',' ')
        #print(l)
        l = f_remove_accents(l)
        x = re.findall("\d\.\s\w\.\s\w+\.$", l)
        y = re.findall("\s\w\.\s\w+$", l)
        if(x):
            print(x)
        if(y):
            print(y)
        
        #   print(l)

[' 0. lecomteanum']
[' o. procerum']
[' o. elatum']
[' o. lecomteanum']


In [104]:
famille =[]
famille_ = []
tailles = []

#for f in folder:
path = 'data/' + 'Flore du Cameroun tome 2 1964 Sapotacees'
print("=============================================================")
print(path)
for i in range(len(os.listdir(path))):
    with open(path+'/'+ str(i+1) +'.txt', 'r') as fp:
        lines = fp.readlines()
        #print("=================")
        for j in range(len(lines)):
            l = lines[j].strip()
            l = l.replace('  ',' ')
            #print(l)
            l = f_remove_accents(l)
            x = re.search("^cle.*espece", l)
            if(x):
                print(i+1)
                print(l)

data/Flore du Cameroun tome 2 1964 Sapotacees
61
clef des especes
92
clef des especes
101
clef des especes
108
clef des especes (i)
122
clef des especes


In [121]:
with open(path+'/'+ str(122) +'.txt', 'r') as fp:
    lines = fp.readlines()
    #print("=================")
    for j in range(len(lines)):
        #print(lines[j])
        l = lines[j].strip()
        l = l.replace('  ',' ')
        #print(l)
        l = f_remove_accents(l)
        x = re.findall("\s\w\.\s\w+$", l)
        if(x):
            print(x)
        #   print(l)

[' d. ogowensis']
[' d. ubangmensis']


# create especes dictionnary

In [13]:
def especes_pages(path):
    pages = []
    for i in range(len(os.listdir(path))):
        with open(path+'/'+ str(i+1) +'.txt', 'r') as fp:
            lines = fp.readlines()
            #print("=================")
            for j in range(len(lines)):
                l = lines[j].strip()
                l = l.replace('  ',' ')
                l = f_remove_accents(l)
                x = re.search("^cle.*espece", l)
                if(x):
                    pages.append(i+1)
                    pages.append(i+2)
                    pages.append(i+3)
    return pages

In [20]:
pages = especes_pages(path)

In [104]:
def get_especes(pages):
    especes = []
    for p in pages:
        with open(path+'/'+ str(p) +'.txt', 'r') as fp:
            lines = fp.readlines()
            for j in range(len(lines)):
                l = lines[j].strip()
                l = l.replace('  ',' ')
                l = f_remove_accents(l)
                x = re.findall("\s\w\.\s\w+$", l)
                if(x):
                    especes.append(x)
    especes = np.array(especes).reshape(-1)
    especes = [esp.strip().split(' ')[1] for esp in especes]
    return especes

def preprocess(line):
    l = line.strip()
    l = l.replace('  ',' ')
    #l = f_remove_accents(l)
    return l

In [64]:
especes = get_especes(pages)

In [107]:
query = "Lecomteanum"
with open(path+'/'+ str(65) +'.txt', 'r') as fp:
    lines = fp.readlines()
    for j in range(len(lines)):
        l = preprocess(lines[j])
        contains = l.find(query) != -1
        if contains:
            x = re.findall("^\d\.\s[A-Z].*", l)
            if(x):
                full_name = x
                print(full_name)

['3. Omphalocarpum Lecomteanum Pierre ex Engl.']


In [118]:
def get_especes_names(especes, path):
    esp_names = []
    for i in range(len(os.listdir(path))):
        with open(path+'/'+ str(i+1) +'.txt', 'r') as fp:
            lines = fp.readlines()
            for j in range(len(lines)):
                l = preprocess(lines[j])
                for esp in especes:
                    contains = l.find(esp.capitalize()) != -1 or l.find(esp) != -1 
                    if contains:
                        x = re.findall("^[1-9]\.\s[A-Z].*", l)
                        if(x):
                            full_name = x[0].strip()[2:]
                            esp_names.append(full_name)
    return esp_names

In [120]:
get_especes_names(especes, path)

[' Manilkara Zenkeri Lecomte ex Aubr. et Pellegr.',
 ' Manilkara Welwitschii (Engl.) Dubard',
 ' Manilkara Letouzei Aubr.',
 ' Omphalocarpum procerum P. Beauv.',
 ' Omphalocarpum elatum Miers',
 ' Omphalocarpum Lecomteanum Pierre ex Engl.',
 ' Omphalocarpum Lecomteanum Pierre ex Engl.',
 ' Synsepalum Letouzei Aubr.',
 ' Synsepalurn dulcificum (Schum.) Bâillon',
 ' Synsepalum Zenkeri Aubr. et Pellegr.',
 ' Synsepalum Batesii (A. Chev.) Aubr. et Pellegr.',
 ' Synsepalum longecuneatum De Wild.',
 ' Vincentella revoluta (Bak.) Pierre',
 ' Vincentella Passargei (Engl.) Aubr.',
 ' Vincentella Brenanii Heine',
 ' G. gigantea',
 ' Gambeya Lacourtiana (De Wild.) Aubr. et Pellegr.',
 ' Gambeya boukokoensis Aubr. et Pellegr.',
 ' Gambeya gigantea (A. Chev.) Aubr. et Pellegr.',
 ' Gambeya albida (G. Don) Aubr. et Pellegr.',
 ' Gambeya subnuda (Bak.) Pierre',
 ' Gambeya africana (Baker) Pierre',
 ' Gambeya perpulchra (Mildbr.) Aubr. et Pellegr.',
 ' Gambeya Beguei (Aubr. et Pellegr.) Aubr. et Pelle

In [111]:
especes

['lecomteanum',
 'procerum',
 'elatum',
 'lecomteanum',
 'subcordatum',
 'letouzei',
 'dulcificum',
 'zenkeri',
 'batesii',
 'longecuneatum',
 'revoluta',
 'passargei',
 'brenanii',
 'lacourtiana',
 'boukokoensis',
 'gigantea',
 'albida',
 'subnuda',
 'africana',
 'perpulchra',
 'beguei',
 'ogowensis',
 'ubangmensis',
 'pruniformis',
 'welwitschii']

In [62]:
import numpy 


especes = np.array(especes).reshape(-1)
especes = [esp.strip().split(' ')[1] for esp in especes]
print(especes)

['lecomteanum', 'procerum', 'elatum', 'lecomteanum', 'subcordatum', 'letouzei', 'dulcificum', 'zenkeri', 'batesii', 'longecuneatum', 'revoluta', 'passargei', 'brenanii', 'lacourtiana', 'boukokoensis', 'gigantea', 'albida', 'subnuda', 'africana', 'perpulchra', 'beguei', 'ogowensis', 'ubangmensis', 'pruniformis', 'welwitschii']


# find organes

In [1]:
organes = [
    "racine",
    "tige",
    "bourgeons",
    "rhizome",
    "tubercule",
    "bulbe",
    "collet",
    "limbe",
    "petiole",
    "sessile",
    "foliole",
    "calice",
    "sepale",
    "calicule",
    "tepale",
    "corolle"
]

In [9]:
def get_organ(line, organ):
    l = line.strip()
    l = l.replace('  ',' ')
    l = f_remove_accents(l)
    start = line.find(organ)
    if start == -1:
        return None
    else:
        return (line.find(organ), line.find(organ) +len(organ))  

In [10]:
famille =[]
famille_ = []
tailles = []

#for f in folder:
path = 'data/' + 'Flore du Cameroun tome 2 1964 Sapotacees'
print("=============================================================")
print(path)
for i in range(len(os.listdir(path))):
    with open(path+'/'+ str(i+1) +'.txt', 'r') as fp:
        lines = fp.readlines()
        #print("=================")
        for j in range(len(lines)):
            l = lines[j].strip()
            l = l.replace('  ',' ')
            #print(l)
            l = f_remove_accents(l)
            ind = get_organ(l, "petiole")
            if(ind != None):
                print(ind)
                print(l)

data/Flore du Cameroun tome 2 1964 Sapotacees
(0, 7)
petiole 1,5-2 cm long i- m. zenkeri
(0, 7)
petiole court, 5-xo mm long. feuilles petites ou relativement
(0, 7)
petiole long, 1,5 a 5 cm :
(22, 29)
long et 5,5 cm large. petiole 2-4 cm long. nervures fines mais visi-
(21, 28)
long sur 8 cm large. petiole 1 -2 cm long -h bord de mer -jh
(21, 28)
long sur 6 cm large. petiole 1-2 cm long. fleurs a pedi-
(0, 7)
petiole court, 5-io mm long. feuilles petites ou relativement petites,
(0, 7)
petiole relativement long ou long :
(19, 26)
obtus et emargine. petiole i,5-2 cm long. environ i5 paires de
(9, 16)
dessous. petiole court, grele, 5-10 mm long.
(0, 7)
petiole, i,5-5 cm de long.
(41, 48)
a l'action des feux de brousses annuels. petiole jusqu'a 4 cm
(46, 53)
gris argente dessous, atteignant i5 x 5,5 cm. petiole 2-3 cm.
(55, 62)
fines, peu visibles, presque droites jusqu'a la marge. petiole
(0, 7)
petiole jusqu'a 2 cm de long.
(57, 64)
sibles en dessous sous le feutrage des poils en navett

In [9]:
import spacy

In [10]:
from nltk.tokenize import sent_tokenize

In [11]:
sentences = sent_tokenize(content)

In [17]:
for s in sentences:
    s = s.strip()
    s = s.replace(" \n", " ")
    s = s.replace("-\n", "")
    print(s)
    print("================")

Feuilles obovées oblongues à oblancéolées, arrondies ou obtusément acuminées au sommet, atténuées et cunéiformes à la base, 12-25 cm long, 4-8 cm large, glabres.
Environ 10 paires de nervures secondaires, saillantes dessous.
Nervilles peu apparentes sur la face supérieure qui est mate.
Pétiole court.
Fleurs à pédicelles de i cm env.
Sépales épais, vert jaunâtre, glabres ou glabrescents.
Corolle blanche, longue de 1,8 cm environ; lobes elliptiques, ciliés sur les bords, n mm; tube 6 mm.
Fleurs hermaphrodites : 5-6 étamines par faisceau, aussi longues que les lobes.
Dans les fleurs femelles, ces étamines sont remplacées par des staminodes subulés.
Ovaire glabre.
Une trentaine de loges uniovulées autour de la partie centrale évidée.
Gros fruits subglobuleux aplat îs, gris brunâtre, environ i5 cm diamètre et 8 cm haut.
Certains pèsent à l'état frais plus de 2 kg.
Ils sont durs extérieurement et contiennent, à l'intérieur d'une pulpe blanchâtre, de nombreuses graines noires plates, elliptiq

# Tesseract OCR

In [ ]:
import os
from PIL import Image
from pdf2image import convert_from_path
import pytesseract

pytesseract.pytesseract.tesseract_cmd =r'C:\Users\NEW-USER\AppData\Local\Tesseract-OCR\tesseract.exe'
poppler_path = r"C:\Users\NEW-USER\AppData\Local\poppler-22.04.0\Library\bin"

In [ ]:
from pdf2image import convert_from_path
from pytesseract import image_to_string
from PIL import Image

In [ ]:
def convert_pdf_img(pdf_file):
    return convert_from_path(pdf_file, poppler_path = poppler_path)
def convert_img_txt(file):
    text = image_to_string(file)
    return text

def get_text_from_any_pdf(pdf_file):
    images = convert_pdf_img(pdf_file)
    final_text = ""
    for pg,img in enumerate(images):
        print(pg)
        final_text += convert_img_txt(img)
    return final_text

In [ ]:
path_pdf = 'data/Flore du Cameroun tome 10 1970 Ombellales.pdf'
pdf1 = get_text_from_any_pdf(path_pdf)
print(pdf1)